In [26]:
import numpy as np
import tensorflow as tf

N_VECTORS = 9
VECTOR_INDEXES = np.arange(N_VECTORS)
VECTORS_VELOCITIES_X = np.array([
    [-1, 0, 1,],
    [-1, 0, 1,],
    [-1, 0, 1,],
]).reshape(-1)
VECTORS_VELOCITIES_Y = np.array([
     [1,  1,  1,],
     [0,  0,  0,],
    [-1, -1, -1,],
]).reshape(-1)

In [4]:
F = np.arange(20).reshape(5, 4)

In [5]:
F

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [18]:
F_wide = np.zeros((F.shape[0] + 2, F.shape[1] + 2), dtype=int)
F_wide[1:-1, 1:-1] = (F)
F_wide[0, 1:-1] = (F[-1, :])
F_wide[-1, 1:-1] = (F[0, :])
F_wide[1:-1, 0] = (F[:, -1])
F_wide[1:-1, -1] = (F[:, 0])
F_wide[0, 0] = (F[-1, -1])
F_wide[-1, -1] = (F[0, 0])
F_wide[0, -1] = (F[-1, 0])
F_wide[-1, 0] = (F[0, -1])

In [19]:
F_wide

array([[19, 16, 17, 18, 19, 16],
       [ 3,  0,  1,  2,  3,  0],
       [ 7,  4,  5,  6,  7,  4],
       [11,  8,  9, 10, 11,  8],
       [15, 12, 13, 14, 15, 12],
       [19, 16, 17, 18, 19, 16],
       [ 3,  0,  1,  2,  3,  0]])

In [45]:
cx = 1
cy = 0
np.roll(F, (cx, cy), axis=(1, 0))

array([[ 3,  0,  1,  2],
       [ 7,  4,  5,  6],
       [11,  8,  9, 10],
       [15, 12, 13, 14],
       [19, 16, 17, 18]])

In [48]:
F_wide[1-cy:F.shape[0] + 1 - cy, 1-cx:F.shape[1] + 1 - cx]

array([[ 3,  0,  1,  2],
       [ 7,  4,  5,  6],
       [11,  8,  9, 10],
       [15, 12, 13, 14],
       [19, 16, 17, 18]])

In [49]:
for i, cx, cy in zip(VECTOR_INDEXES, VECTORS_VELOCITIES_X, VECTORS_VELOCITIES_Y):
    rolled_F = np.roll(F, (cx, cy), axis=(1, 0))
    sliced_F = F_wide[1-cy:F.shape[0] + 1 - cy, 1-cx:F.shape[1] + 1 - cx]
    assert np.all(rolled_F == sliced_F)

## Experiments with large scale F

In [59]:
LENGTH_Y = 100
LENGTH_X = 400
F = np.random.rand(*(LENGTH_Y, LENGTH_X, N_VECTORS))
wide_F = np.zeros((F.shape[0] + 2, F.shape[1] + 2, 9))
wide_F[1:-1, 1:-1, :] = (F)
wide_F[0, 1:-1, :] = (F[-1, :, :])
wide_F[-1, 1:-1, :] = (F[0, :, :])
wide_F[1:-1, 0, :] = (F[:, -1, :])
wide_F[1:-1, -1, :] = (F[:, 0, :])
wide_F[0, 0, :] = (F[-1, -1, :])
wide_F[-1, -1, :] = (F[0, 0, :])
wide_F[0, -1, :] = (F[-1, 0, :])
wide_F[-1, 0, :] = (F[0, -1, :])

for i, cx, cy in zip(VECTOR_INDEXES, VECTORS_VELOCITIES_X, VECTORS_VELOCITIES_Y):
    rolled_F = np.roll(F[:, :, i], (cx, cy), axis=(1, 0))
    sliced_F = wide_F[:, :, i][1-cy:F.shape[0] + 1 - cy, 1-cx:F.shape[1] + 1 - cx]
    assert np.all(rolled_F == sliced_F)

In [81]:
rho = np.sum(F, 2)
ux  = np.sum(F * VECTORS_VELOCITIES_X, 2) / rho
uy  = np.sum(F * VECTORS_VELOCITIES_Y, 2) / rho
u = np.sqrt(ux ** 2 + uy ** 2)

vorticity = (
    (np.roll(ux, -1, axis=0) - np.roll(ux, 1, axis=0)) - 
    (np.roll(uy, -1, axis=1) - np.roll(uy, 1, axis=1))
)

In [68]:
def enlarge_array(arr):
    arr_large = np.zeros((arr.shape[0] + 2, arr.shape[1] + 2))
    arr_large[1:-1, 1:-1] = (arr)
    arr_large[0, 1:-1] = (arr[-1, :])
    arr_large[-1, 1:-1] = (arr[0, :])
    arr_large[1:-1, 0] = (arr[:, -1])
    arr_large[1:-1, -1] = (arr[:, 0])
    arr_large[0, 0] = (arr[-1, -1])
    arr_large[-1, -1] = (arr[0, 0])
    arr_large[0, -1] = (arr[-1, 0])
    arr_large[-1, 0] = (arr[0, -1])
    return arr_large

In [80]:
ux_l = enlarge_array(ux)
uy_l = enlarge_array(uy)

vorticity_l = (
    (ux_l[2:, 1:-1] - ux_l[0:-2, 1:-1]) -
    (uy_l[1:-1, 2:] - uy_l[1:-1, 0:-2])
)

In [82]:
assert np.all(vorticity == vorticity_l)